**Pytorch BERT baseline**

In this version, I convert https://www.kaggle.com/akensert/bert-base-tf2-0-minimalistic into pytorch version

**Please upvote the kernel if you find it helpful**

### Install HuggingFace transformers & sacremoses dependency

As we are not allowed to use internet I've created required datasets and commands to setup Hugging Face Transformers setup in offline mode. You can find the required github codebases in the datasets.

* sacremoses dependency - https://www.kaggle.com/axel81/sacremoses
* transformers - https://www.kaggle.com/axel81/transformers

In [1]:
# !pip install ./sacremoses/sacremoses-master/
# !pip install ./transformers/transformers-master/
STRIDE = 1
def is_jupyter():
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return True
        
    except:
        return False

### Required Imports

I've added imports that will be used in training too

In [2]:
from auc import MyAUCCallback
from sklearn.utils import shuffle
from datetime import datetime
import pandas as pd
pd.set_option('display.max.columns', 500)
import numpy as np
import os
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-10.1/lib64'
import matplotlib.pyplot as plt
import gc
from shutil import copyfile
from catalyst.dl import SupervisedRunner, AlchemyLogger, CriterionCallback
from catalyst.dl.callbacks.metrics import AUCCallback
from torch.utils.data import DataLoader, SubsetRandomSampler,Dataset
batch_size = 12
token = "d1dd16f08d518293bcbeddd313b49aa4"
DATA_DIR = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'

/home/kb/jig_env/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [3]:
if os.uname()[1] == 'kb-Z370P-D3':
    # desktop
    LOG_PATH = '/media/ssd/logs/jigsaw'
    SERVER = False
    print('Working on desktop')
elif os.uname()[1] == 'kb-server':
    # server
    LOG_PATH = '/home/kb/logs/jigsaw'
    SERVER = True
    print('Working on server')
else:
    raise Exception('which hostname???')
    

Working on desktop


In [4]:
# df_train_toxic = pd.read_csv(DATA_DIR+'jigsaw-toxic-comment-train.csv')
# df_train_toxic.head()

In [5]:
# df_train_bias = pd.read_csv(DATA_DIR + 'jigsaw-unintended-bias-train.csv')
# df_train_bias.head()

In [6]:
# len(df_train_toxic[df_train_toxic['toxic']==1]),len(df_train_toxic[df_train_toxic['toxic']==0])
# len(df_train_bias[df_train_bias['toxic']<0.1])/7,len(df_train_bias[df_train_bias['toxic']>0.80])

In [7]:
# df_train_bias_pos = df_train_bias[df_train_bias['toxic']>0.8].reset_index(drop=True)
# df_train_bias_neg = shuffle(df_train_bias[df_train_bias['toxic']<0.1].reset_index(drop=True)).reset_index(drop=True).iloc[::7]
# df_train_bias =  df_train_bias_pos.append(df_train_bias_neg).reset_index(drop=True)

In [8]:
df_valid = shuffle(pd.read_csv(DATA_DIR + 'validation.csv'))
df_valid.head()

,id,comment_text,lang,toxic
7005,7005,"Merhaba İloşda, bir başka kullanıcıya mesaj at...",tr,0
3262,3262,", bir çok yerde karşılaştığım bu fotoğraf sizi...",tr,0
1322,1322,ben metayı karl marx gibi değerlendirmiyorum....,tr,1
2773,2773,He borrado esa sección porque parece una b...,es,0
5889,5889,"Per favore Blackcat, smetti di ricorrere ai tu...",it,0


In [9]:
df_valid.groupby(['lang', 'toxic']).count()

id  comment_text
lang toxic                    
es   0      2078          2078
     1       422           422
it   0      2012          2012
     1       488           488
tr   0      2680          2680
     1       320           320

In [10]:
df_test = pd.read_csv(DATA_DIR + 'test.csv')
df_test.head()

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr


In [11]:
df_test.groupby(['lang']).count()

,id,content
lang,,
es,8438,8438
fr,10920,10920
it,8494,8494
pt,11012,11012
ru,10948,10948
tr,14000,14000


In [12]:
len(df_test)

63812

In [13]:
df_sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
df_sub.head()

,id,toxic
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [14]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [15]:
target_columns = 'toxic'

### Define dataset

In [16]:
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import time
from torch.optim import lr_scheduler

import torch
from tqdm import tqdm
#import torch.utils.data as data
from torchvision import datasets, models, transforms
from transformers import *
import random
from math import floor, ceil
from sklearn.model_selection import GroupKFold

MAX_LEN = 512
SEP_TOKEN_ID = 102

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        self.df = df
        if train_mode:
            self.labels = df.toxic.values
            
        self.train_mode = train_mode
        self.labeled = labeled
        #self.tokenizer = DistilBertTokenizer.from_pretrained('bert-base-uncased')
#         self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
        #distilbert-base-multilingual-cased

    def __getitem__(self, index):
        row = self.df.iloc[index]
        token_ids = self.get_token_ids(row)
        
        if self.labeled:
            labels = self.get_label(row)
            return {'features': token_ids, 'targets': labels}

        else:
            return {'features': token_ids}

    def __len__(self):
        return len(self.df)

    def trim_input(self, text, max_sequence_length=MAX_LEN):
        t = self.tokenizer.tokenize(text)
        t_len = len(t)

        if t_len + 2 > max_sequence_length:

            t_new_len = int(max_sequence_length) - 2

            t = t[:t_new_len]

        return t
        
    def get_token_ids(self, row):
        t_tokens = self.trim_input(row.comment_text)

#         tokens = ['[CLS]'] + t_tokens  + ['[SEP]']+ t_tokens[-1::-1]+ ['[SEP]']
        tokens = ['[CLS]'] + t_tokens  + ['[SEP]']
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        
        if len(token_ids) < MAX_LEN:
            token_ids += [0] * (MAX_LEN - len(token_ids))
            
        ids = torch.tensor(token_ids)
        
        return ids

    def get_label(self, row):
#         label = torch.tensor(row[target_columns].astype(np.long))
        label = np.round(row[target_columns])
        return torch.tensor([1-label, label]).float()
    
    def collate_fn(self, batch):
        token_ids = torch.stack([x[0] for x in batch])

        if self.labeled:
            labels = torch.stack([x[1] for x in batch])
            return {'features': token_ids, 'targets': labels}
        else:
            return {'features': token_ids}

/home/kb/jig_env/lib/python3.7/site-packages/torchvision/extension.py:11: ResourceWarning:

unclosed file <_io.BufferedReader name='/home/kb/jig_env/lib/python3.7/site-packages/torchvision/_C.so'>



## Build Model

In [17]:
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F


class QuestModel(nn.Module):
    def __init__(self, n_classes=2):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased') 
#         self.bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.bert_model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
    
        self.fc = nn.Linear(768, n_classes)

    def forward(self, ids):
        attention_mask = (ids > 0)
        layers = self.bert_model(input_ids=ids, attention_mask=attention_mask)
        
        out = F.dropout(layers[-1][:, 0, :], p=0.2, training=self.training)
        logit = self.fc(out)#.unsqueeze(1)
        return logit #, 'for_auc': logit[:, 1]}#[:,1]
    


In [18]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
        callback_get_label func: a callback-like function which takes two arguments - dataset and index
    """

    def __init__(self, dataset, indices=None, num_samples=None, callback_get_label=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices

        # define custom callback
        self.callback_get_label = callback_get_label

        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        if self.callback_get_label:
            return self.callback_get_label(dataset, idx)
        else:
            dataset.labels[idx]
#             raise NotImplementedError
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [19]:

def get_loaders(df_valid, val_pt='es', to_balance=True, shuffle_before=True, inv=False):
    if SERVER:
        workers=1
    else:
        workers = 6
        

    if shuffle_before:
        df_valid = shuffle(df_valid)
    

    train_loader = DataLoader(
        QuestDataset(df_valid[df_valid['lang']!=val_pt].iloc[::STRIDE], train_mode=True),
        num_workers=workers,
#         sampler=ImbalancedDatasetSampler(train_dataset) if to_balance else None,
        batch_size=batch_size,
    )
    valid_loader = DataLoader(
        QuestDataset(df_valid[df_valid['lang']==val_pt].iloc[::STRIDE], train_mode=False),
        num_workers=workers,
        batch_size=batch_size,
    )    
           
    loaders = {}
    if not inv:
        loaders['train'] = train_loader
        loaders['valid'] = valid_loader
    else:
        loaders['train'] = valid_loader
        loaders['valid'] = train_loader
        

    
    return loaders


In [20]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
CHECKPOINT = './checkpoints/from_v4.pth'
project = "jigsaw_v6_distilbert_finetune_v4_with_auc_main_metric"
num_epochs = 25

group = datetime.now().strftime("%m_%d_%Y__%H_%M_%S")

    
if SERVER:
    group = f'{group}_server'
    
if STRIDE > 1:
    group = f'{group}_str{STRIDE}'
#['es', 'it', 'tr']
variants = {
    '_es': 'es',
    '_it': 'it',
    '_tr': 'tr'
}
gradient_accumulation_steps = 1

lr = 3e-5#0.0001
group += f'_lr{lr}'

group = group.replace('.', '')

runner = SupervisedRunner(input_key=('features'), input_target_key=('targets'), output_key=('logits'))

for inv in (False, True):
    for exp in variants.keys():
        experiment = 'train'+exp if inv else 'val'+exp
        torch.cuda.empty_cache()

        logdir = f"{LOG_PATH}/{project}/{group}/{experiment}"

        model = QuestModel(2)

        checkpoint = torch.load(CHECKPOINT)#, map_location=device)   

        model.load_state_dict(checkpoint['model_state_dict'])
        del checkpoint
    #     model = model.to(device)

        loaders = get_loaders(df_valid, variants[exp], to_balance=True, inv=inv)


        t_total = len(loaders['train'])//gradient_accumulation_steps*num_epochs
        warmup_proportion = 0.01
        num_warmup_steps = t_total * warmup_proportion

        criterion = torch.nn.BCEWithLogitsLoss()
    #     criterion = torch.nn.CrossEntropyLoss()
    #     criterion = torch.nn.BCELoss()
        optimizer = AdamW(model.parameters(), lr = lr)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total) 
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.25)
        print(f'----------------Experiment: {experiment}')

        runner.train(
            model=model,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            loaders=loaders,
            logdir=logdir,
            num_epochs=num_epochs,
            verbose=True,
            distributed=False if is_jupyter() else True,
            callbacks=[
                AlchemyLogger(
                        token=token, # your Alchemy token
                        project=project,
                        experiment=experiment,
                        group=group,
                    ),
                MyAUCCallback()
    #             AUCCallback(input_key = 'targets',
    #                                 output_key = 'logits',
    #                                 prefix = 'auc',
    #                                 class_names = None,
    #                                 num_classes = 2,
    #                                 activation = 'Sigmoid',)
            ],
            main_metric='auc/_mean',
            minimize_metric=False

        )

----------------Experiment: val_es
1/25 * Epoch (train): 100% 459/459 [01:39<00:00,  4.59it/s, loss=0.002]
1/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.74it/s, loss=0.014]
[2020-04-19 18:24:07,694] 
1/25 * Epoch 1 (_base): lr=2.614e-07 | momentum=0.9000
1/25 * Epoch 1 (train): auc/_mean=0.7970 | auc/class_0=0.7970 | loss=0.7177
1/25 * Epoch 1 (valid): auc/_mean=0.8229 | auc/class_0=0.8229 | loss=0.7612


/home/kb/jig_env/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:224: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



2/25 * Epoch (train): 100% 459/459 [01:40<00:00,  4.57it/s, loss=0.038]
2/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.59it/s, loss=0.007]
[2020-04-19 18:26:13,137] 
2/25 * Epoch 2 (_base): lr=5.229e-07 | momentum=0.9000
2/25 * Epoch 2 (train): auc/_mean=0.7819 | auc/class_0=0.7819 | loss=0.3901
2/25 * Epoch 2 (valid): auc/_mean=0.8431 | auc/class_0=0.8431 | loss=0.3850
3/25 * Epoch (train): 100% 459/459 [01:39<00:00,  4.59it/s, loss=0.054]
3/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.74it/s, loss=0.015]
[2020-04-19 18:28:46,502] 
3/25 * Epoch 3 (_base): lr=7.843e-07 | momentum=0.9000
3/25 * Epoch 3 (train): auc/_mean=0.8653 | auc/class_0=0.8653 | loss=0.2964
3/25 * Epoch 3 (valid): auc/_mean=0.8498 | auc/class_0=0.8498 | loss=0.3680
4/25 * Epoch (train): 100% 459/459 [01:40<00:00,  4.56it/s, loss=0.047]
4/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.19it/s, loss=0.023]
[2020-04-19 18:31:20,914] 
4/25 * Epoch 4 (_base): lr=1.046e-06 | momentum=0.9000
4/25 * Epoch 4 (tr

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connectionpool.py", line 376, in _make_request
    self._validate_conn(conn)
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connectionpool.py", line 994, in _validate_c

6/25 * Epoch (train): 100% 459/459 [01:39<00:00,  4.62it/s, loss=0.027]
6/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.75it/s, loss=0.025]
[2020-04-19 18:36:30,050] 
6/25 * Epoch 6 (_base): lr=1.569e-06 | momentum=0.9000
6/25 * Epoch 6 (train): auc/_mean=0.9236 | auc/class_0=0.9236 | loss=0.2309
6/25 * Epoch 6 (valid): auc/_mean=0.8574 | auc/class_0=0.8574 | loss=0.3602
7/25 * Epoch (train): 100% 459/459 [01:39<00:00,  4.59it/s, loss=0.048]
7/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.10it/s, loss=0.023]
[2020-04-19 18:39:04,000] 
7/25 * Epoch 7 (_base): lr=1.830e-06 | momentum=0.9000
7/25 * Epoch 7 (train): auc/_mean=0.9392 | auc/class_0=0.9392 | loss=0.2084
7/25 * Epoch 7 (valid): auc/_mean=0.8568 | auc/class_0=0.8568 | loss=0.3600
8/25 * Epoch (train): 100% 459/459 [01:39<00:00,  4.60it/s, loss=0.015]
8/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.70it/s, loss=0.021]
[2020-04-19 18:41:20,925] 
8/25 * Epoch 8 (_base): lr=2.092e-06 | momentum=0.9000
8/25 * Epoch 8 (tr

...

23/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.69it/s, loss=0.562]
[2020-04-19 20:12:31,411] 
23/25 * Epoch 48 (_base): lr=6.013e-06 | momentum=0.9000
23/25 * Epoch 48 (train): auc/_mean=0.9997 | auc/class_0=0.9997 | loss=0.0145
23/25 * Epoch 48 (valid): auc/_mean=0.7950 | auc/class_0=0.7950 | loss=1.0549
24/25 * Epoch (train): 100% 459/459 [01:39<00:00,  4.60it/s, loss=0.002]    
24/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.67it/s, loss=0.402]
[2020-04-19 20:14:46,448] 
24/25 * Epoch 49 (_base): lr=6.275e-06 | momentum=0.9000
24/25 * Epoch 49 (train): auc/_mean=0.9992 | auc/class_0=0.9992 | loss=0.0196
24/25 * Epoch 49 (valid): auc/_mean=0.7963 | auc/class_0=0.7963 | loss=1.1273
25/25 * Epoch (train): 100% 459/459 [01:39<00:00,  4.61it/s, loss=0.001]    
25/25 * Epoch (valid): 100% 209/209 [00:14<00:00, 14.68it/s, loss=0.005]
[2020-04-19 20:17:01,350] 
25/25 * Epoch 50 (_base): lr=6.536e-06 | momentum=0.9000
25/25 * Epoch 50 (train): auc/_mean=0.9996 | auc/class_0=0.9996

19/25 * Epoch (train): 100% 417/417 [01:30<00:00,  4.63it/s, loss=0.339]    
19/25 * Epoch (valid): 100% 250/250 [00:16<00:00, 14.75it/s, loss=0.263]
[2020-04-19 20:59:07,093] 
19/25 * Epoch 69 (_base): lr=5.468e-06 | momentum=0.9000
19/25 * Epoch 69 (train): auc/_mean=0.9992 | auc/class_0=0.9992 | loss=0.0282
19/25 * Epoch 69 (valid): auc/_mean=0.8551 | auc/class_0=0.8551 | loss=0.4375
20/25 * Epoch (train): 100% 417/417 [01:30<00:00,  4.63it/s, loss=0.003]    
20/25 * Epoch (valid): 100% 250/250 [00:16<00:00, 14.77it/s, loss=0.223]
[2020-04-19 21:01:15,725] 
20/25 * Epoch 70 (_base): lr=5.755e-06 | momentum=0.9000
20/25 * Epoch 70 (train): auc/_mean=0.9987 | auc/class_0=0.9987 | loss=0.0362
20/25 * Epoch 70 (valid): auc/_mean=0.8761 | auc/class_0=0.8761 | loss=0.3085
21/25 * Epoch (train): 100% 417/417 [01:30<00:00,  4.63it/s, loss=0.003]    
21/25 * Epoch (valid): 100% 250/250 [00:16<00:00, 14.74it/s, loss=0.264]
[2020-04-19 21:03:24,319] 
21/25 * Epoch 71 (_base): lr=6.043e-06 | mo

15/25 * Epoch (train): 100% 209/209 [00:46<00:00,  4.50it/s, loss=0.002]
15/25 * Epoch (valid): 100% 459/459 [00:32<00:00, 14.15it/s, loss=0.011]
[2020-04-19 21:37:54,337] 
15/25 * Epoch 90 (_base): lr=8.612e-06 | momentum=0.9000
15/25 * Epoch 90 (train): auc/_mean=0.9985 | auc/class_0=0.9985 | loss=0.0419
15/25 * Epoch 90 (valid): auc/_mean=0.8118 | auc/class_0=0.8118 | loss=0.5803
16/25 * Epoch (train): 100% 209/209 [00:46<00:00,  4.50it/s, loss=0.243]  
16/25 * Epoch (valid): 100% 459/459 [00:32<00:00, 14.12it/s, loss=0.014]
[2020-04-19 21:39:37,057] 
16/25 * Epoch 91 (_base): lr=9.187e-06 | momentum=0.9000
16/25 * Epoch 91 (train): auc/_mean=0.9992 | auc/class_0=0.9992 | loss=0.0277
16/25 * Epoch 91 (valid): auc/_mean=0.8076 | auc/class_0=0.8076 | loss=0.6296
17/25 * Epoch (train): 100% 209/209 [00:46<00:00,  4.48it/s, loss=0.002]    
17/25 * Epoch (valid): 100% 459/459 [00:32<00:00, 14.01it/s, loss=0.024]
[2020-04-19 21:41:20,878] 
17/25 * Epoch 92 (_base): lr=9.761e-06 | momentum

11/25 * Epoch (train): 100% 209/209 [00:46<00:00,  4.52it/s, loss=0.134]
11/25 * Epoch (valid): 100% 459/459 [00:32<00:00, 14.26it/s, loss=0.013]
[2020-04-19 22:15:00,439] 
11/25 * Epoch 111 (_base): lr=6.316e-06 | momentum=0.9000
11/25 * Epoch 111 (train): auc/_mean=0.9939 | auc/class_0=0.9939 | loss=0.0823
11/25 * Epoch 111 (valid): auc/_mean=0.8321 | auc/class_0=0.8321 | loss=0.5275
12/25 * Epoch (train): 100% 209/209 [00:46<00:00,  4.52it/s, loss=0.003]
12/25 * Epoch (valid): 100% 459/459 [00:32<00:00, 14.22it/s, loss=0.036]
[2020-04-19 22:16:41,986] 
12/25 * Epoch 112 (_base): lr=6.890e-06 | momentum=0.9000
12/25 * Epoch 112 (train): auc/_mean=0.9961 | auc/class_0=0.9961 | loss=0.0620
12/25 * Epoch 112 (valid): auc/_mean=0.8461 | auc/class_0=0.8461 | loss=0.4936
13/25 * Epoch (train): 100% 209/209 [00:46<00:00,  4.49it/s, loss=0.001]
13/25 * Epoch (valid): 100% 459/459 [00:32<00:00, 14.20it/s, loss=0.012]
[2020-04-19 22:18:23,836] 
13/25 * Epoch 113 (_base): lr=7.464e-06 | momentu

7/25 * Epoch (train): 100% 250/250 [00:54<00:00,  4.58it/s, loss=0.006]
7/25 * Epoch (valid): 100% 417/417 [00:28<00:00, 14.61it/s, loss=1.190]
[2020-04-19 22:51:28,647] 
7/25 * Epoch 132 (_base): lr=3.360e-06 | momentum=0.9000
7/25 * Epoch 132 (train): auc/_mean=0.9871 | auc/class_0=0.9871 | loss=0.0873
7/25 * Epoch 132 (valid): auc/_mean=0.8185 | auc/class_0=0.8185 | loss=0.5360
8/25 * Epoch (train): 100% 250/250 [00:54<00:00,  4.56it/s, loss=0.002]
8/25 * Epoch (valid): 100% 417/417 [00:28<00:00, 14.65it/s, loss=1.591]
[2020-04-19 22:53:12,864] 
8/25 * Epoch 133 (_base): lr=3.840e-06 | momentum=0.9000
8/25 * Epoch 133 (train): auc/_mean=0.9931 | auc/class_0=0.9931 | loss=0.0656
8/25 * Epoch 133 (valid): auc/_mean=0.8101 | auc/class_0=0.8101 | loss=0.6369
9/25 * Epoch (train): 100% 250/250 [00:54<00:00,  4.56it/s, loss=0.001]    
9/25 * Epoch (valid): 100% 417/417 [00:28<00:00, 14.56it/s, loss=1.942]
[2020-04-19 22:54:57,116] 
9/25 * Epoch 134 (_base): lr=4.320e-06 | momentum=0.9000
